# **INTRODUCTIONS**

---

ECDC (European Centre for Disease Prevention and Control) is the agency that records data of COVID-19 in whole EU.
This specific workflow gathers all the recorded cases throughout the EU.

2022-04: Currently a fix is needed because the sources format has been changed.

1. **[Tables](#TABLES)**
2. **[Views](#VIEWS)**
3. **[Stored Procedures](#stored=procedures)**


# **DEPENDENCIES**
---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
        "src/dataflows/rivm_number_municipality.ipynb"
    ]
}
```

# **TABLES**
---

### STAGINGS

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.
IF NOT EXISTS(SELECT * FROM sys.sequences WHERE object_id = OBJECT_ID(N'[dbo].[SEQ_VWSSTAGE_ECDC_CASES]') AND type = 'SO')
CREATE SEQUENCE SEQ_VWSSTAGE_ECDC_CASES
  START WITH 1
  INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[ECDC_CASES]'))
BEGIN
CREATE TABLE VWSSTAGE.ECDC_CASES(
  [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSSTAGE_ECDC_CASES]),
  [COUNTRY] VARCHAR(100) NULL,
  [COUNTRY_CODE] VARCHAR(100) NULL,
  [CONTINENT] VARCHAR(100) NULL,
  [POPULATION] VARCHAR(100) NULL,
  [INDICATOR] VARCHAR(100) NULL,
  [WEEKLY_COUNT] VARCHAR(100) NULL,
  [YEAR_WEEK] VARCHAR(100) NULL,
  [RATE_14_DAY] VARCHAR(100) NULL,
  [CUMULATIVE_COUNT] VARCHAR(100) NULL,
  [SOURCE] VARCHAR(100) NULL,
  [DATE_LAST_INSERTED]  DATETIME DEFAULT GETDATE()
);
END

IF NOT EXISTS(SELECT * FROM sys.indexes WHERE NAME='IX_STAGE_ECDC_CASES')
    CREATE NONCLUSTERED INDEX IX_STAGE_ECDC_CASES
    ON VWSSTAGE.ECDC_CASES(DATE_LAST_INSERTED)
    INCLUDE (
            [COUNTRY], 
            [COUNTRY_CODE],
            [CONTINENT],
            [POPULATION],
            [INDICATOR],
            [WEEKLY_COUNT],
            [YEAR_WEEK],
            [RATE_14_DAY],
            [CUMULATIVE_COUNT],
            [SOURCE]
    );
GO

### INTERMEDIATES

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.
IF NOT EXISTS(SELECT * FROM sys.sequences WHERE object_id = OBJECT_ID(N'[dbo].[SEQ_VWSINTER_ECDC_CASES]') AND type = 'SO')
CREATE SEQUENCE SEQ_VWSINTER_ECDC_CASES
  START WITH 1
  INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[ECDC_CASES]'))
BEGIN
CREATE TABLE VWSINTER.ECDC_CASES(
  [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSINTER_ECDC_CASES]),
  [COUNTRY] VARCHAR(100) NULL,
  [COUNTRY_CODE] VARCHAR(100) NULL,
  [CONTINENT] VARCHAR(100) NULL,
  [POPULATION] BIGINT NULL,
  [INDICATOR] VARCHAR(100) NULL,
  [WEEKLY_COUNT] BIGINT NULL,
  [YEAR_WEEK] VARCHAR(100) NULL,
  [WEEK_START] DATE NULL,
  [WEEK_END] DATE NULL,
  [RATE_14_DAY] FLOAT NULL,
  [CUMULATIVE_COUNT] BIGINT NULL,
  [SOURCE] VARCHAR(100) NULL,
  [DATE_LAST_INSERTED]  DATETIME DEFAULT GETDATE()
);
END

IF NOT EXISTS(SELECT * FROM sys.indexes WHERE NAME='IX_INTER_ECDC_CASES')
    CREATE NONCLUSTERED INDEX IX_INTER_ECDC_CASES
    ON VWSINTER.ECDC_CASES(DATE_LAST_INSERTED)
    INCLUDE (
            [COUNTRY], 
            [COUNTRY_CODE],
            [CONTINENT],
            [POPULATION],
            [INDICATOR],
            [WEEKLY_COUNT],
            [YEAR_WEEK],
            [WEEK_START],
            [WEEK_END],
            [RATE_14_DAY],
            [CUMULATIVE_COUNT],
            [SOURCE]
    );
GO

### DESTINATIONS

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.
IF NOT EXISTS(SELECT * FROM sys.sequences WHERE object_id = OBJECT_ID(N'[dbo].[SEQ_VWSDEST_ECDC_CASES]') AND type = 'SO')
CREATE SEQUENCE SEQ_VWSDEST_ECDC_CASES
  START WITH 1
  INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[ECDC_CASES]'))
BEGIN
CREATE TABLE VWSDEST.ECDC_CASES(
  [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSDEST_ECDC_CASES]),
  [COUNTRY] VARCHAR(100) NULL,
  [COUNTRY_CODE] VARCHAR(100) NULL,
  [CONTINENT] VARCHAR(100) NULL,
  [POPULATION] BIGINT NULL,
  [INDICATOR] VARCHAR(100) NULL,
  [YEAR_WEEK] VARCHAR(100) NULL,
  [WEEK_START] DATE NULL,
  [WEEK_END] DATE NULL,
  [RATE_14_DAY] FLOAT NULL,
  [CUMULATIVE_COUNT] BIGINT NULL,
  [SOURCE] VARCHAR(100) NULL,
  [INFECTED_TOTAL] INT NULL,
  [INFECTED_TOTAL_AVERAGE] DECIMAL(19,3) NULL,
  [INFECTED_PER_100K] DECIMAL(19,3) NULL,
  [INFECTED_PER_100K_AVERAGE]  DECIMAL(19,3) NULL,
  [DATE_LAST_INSERTED]  DATETIME DEFAULT GETDATE()
);
END

IF NOT EXISTS(SELECT * FROM sys.indexes WHERE NAME='IX_DEST_ECDC_CASES')
    CREATE NONCLUSTERED INDEX IX_DEST_ECDC_CASES
    ON VWSDEST.ECDC_CASES(DATE_LAST_INSERTED)
    INCLUDE (
            [COUNTRY], 
            [COUNTRY_CODE],
            [CONTINENT],
            [POPULATION],
            [INDICATOR],
            [YEAR_WEEK],
            [WEEK_START],
            [WEEK_END],
            [RATE_14_DAY],
            [CUMULATIVE_COUNT],
            [SOURCE],
            [INFECTED_TOTAL],
            [INFECTED_TOTAL_AVERAGE],
            [INFECTED_PER_100K],
            [INFECTED_PER_100K_AVERAGE]
    );
GO

# **VIEWS**
---

### CASES

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW [VWSDEST].[V_ECDC_CASES] AS
WITH NL_CTE AS (
SELECT 
       'Netherlands' AS [COUNTRY]
      ,'NLD'         AS [COUNTRY_CODE]
      ,[POPULATION]
      ,DBO.CONVERT_DATETIME_TO_UNIX([WEEK_START])   AS [DATE_START_UNIX]
      ,DBO.CONVERT_DATETIME_TO_UNIX([WEEK_END])     AS [DATE_END_UNIX]
      ,[TOT_INFECTED]                               AS [INFECTED]
      ,ROUND([TOT_INFECTED_RATE] ,1)                           AS [INFECTED_TOTAL_AVERAGE]
      ,ROUND([REL_INFECTED]      ,1)                         AS [INFECTED_PER_100K]
      ,ROUND([REL_INFECTED_RATE] ,1)                         AS [INFECTED_PER_100K_AVERAGE]
      ,dbo.CONVERT_DATETIME_TO_UNIX([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
  FROM [VWSDEST].[POSITIVE_TESTED_PEOPLE_WEEKLY]
  WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[POSITIVE_TESTED_PEOPLE_WEEKLY])
)
,
INTERNATIONAL_CTE AS (
SELECT 

       [COUNTRY]
      ,[COUNTRY_CODE]
      ,[POPULATION]
      ,DBO.CONVERT_DATETIME_TO_UNIX([WEEK_START]) AS [DATE_START_UNIX]
      ,DBO.CONVERT_DATETIME_TO_UNIX([WEEK_END])  AS [DATE_END_UNIX]
      ,[INFECTED_TOTAL] AS [INFECTED]
      ,ROUND([INFECTED_TOTAL_AVERAGE],1)        AS [INFECTED_TOTAL_AVERAGE]
      ,ROUND([INFECTED_PER_100K],1)             AS [INFECTED_PER_100K]
      ,ROUND([INFECTED_PER_100K_AVERAGE],1)     AS [INFECTED_PER_100K_AVERAGE]
      ,dbo.CONVERT_DATETIME_TO_UNIX([DATE_LAST_INSERTED]) AS [DATE_OF_INSERTION_UNIX]
  FROM [VWSDEST].[ECDC_CASES]
  WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[ECDC_CASES])
  AND COUNTRY_CODE <> 'NLD'
)
,
--Select all rows
COMBINED_CTE AS (
        SELECT * FROM NL_CTE
        UNION ALL 
        SELECT * FROM INTERNATIONAL_CTE
)

SELECT * 

FROM COMBINED_CTE
--Make sure the last dates of NL and international are aligned
WHERE DATE_END_UNIX <= (SELECT MAX([DATE_END_UNIX]) FROM NL_CTE)
  AND DATE_END_UNIX <= (SELECT MAX([DATE_END_UNIX]) FROM INTERNATIONAL_CTE)
GO


# **Stored Procedures**
---

### STAGING TO INTER

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER PROCEDURE [dbo].[SP_ECDC_CASES_INTER]
AS
BEGIN
    INSERT INTO VWSINTER.ECDC_CASES
    (
        [COUNTRY],
        [COUNTRY_CODE],
        [CONTINENT],
        [POPULATION],
        [INDICATOR],
        [WEEKLY_COUNT],
        [YEAR_WEEK],
        [WEEK_START],
        [WEEK_END],
        [RATE_14_DAY],
        [CUMULATIVE_COUNT],
        [SOURCE]
    )
     SELECT
        [COUNTRY],
        [COUNTRY_CODE],
        [CONTINENT],
        CAST([POPULATION] AS BIGINT) AS [POPULATION],
        [INDICATOR],
        CASE 
            WHEN [WEEKLY_COUNT] = 'NA'
            THEN CAST('' AS FLOAT)
            ELSE CAST([WEEKLY_COUNT] AS BIGINT) 
        END AS [WEEKLY_COUNT],  
        [YEAR_WEEK],
        CAST([dbo].[CONVERT_ISO_WEEK_TO_DATETIME](LEFT(YEAR_WEEK,4),RIGHT(YEAR_WEEK,2)) AS DATE) AS [WEEK_START],
        CAST([dbo].[WEEK_END]([dbo].[CONVERT_ISO_WEEK_TO_DATETIME](LEFT(YEAR_WEEK,4),RIGHT(YEAR_WEEK,2))) AS DATE ) AS [WEEK_END],
        CASE 
            WHEN [RATE_14_DAY] = 'NA'
            THEN CAST('' AS FLOAT)
            ELSE CAST([RATE_14_DAY] AS FLOAT) 
        END AS [RATE_14_DAY],       
        CASE 
            WHEN [CUMULATIVE_COUNT] = 'NA'
            THEN NULL
            ELSE CAST([CUMULATIVE_COUNT] AS BIGINT) 
        END AS [CUMULATIVE_COUNT],
        [SOURCE]
    FROM 
       VWSSTAGE.ECDC_CASES
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) from VWSSTAGE.ECDC_CASES)
END;
GO


### INTER TO DEST

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER PROCEDURE DBO.SP_ECDC_CASES
AS
BEGIN
    INSERT INTO VWSDEST.ECDC_CASES
    (
        [COUNTRY],
        [COUNTRY_CODE],
        [CONTINENT],
        [POPULATION],
        [INDICATOR],
        [YEAR_WEEK],
        [WEEK_START],
        [WEEK_END],
        [RATE_14_DAY],
        [CUMULATIVE_COUNT],
        [SOURCE],
        [INFECTED_TOTAL],
        [INFECTED_TOTAL_AVERAGE],
        [INFECTED_PER_100K],
        [INFECTED_PER_100K_AVERAGE]
    )
    SELECT
        [COUNTRY],
        [COUNTRY_CODE],
        [CONTINENT],
        [POPULATION],
        [INDICATOR],
        [YEAR_WEEK],
        [WEEK_START],
        [WEEK_END],
        [RATE_14_DAY],
        [CUMULATIVE_COUNT],
        [SOURCE],
        --Total refers to week totals. Average refers to daily averages, hence the weektotals / 7
        [WEEKLY_COUNT] AS [INFECTED_TOTAL],
        (CAST([WEEKLY_COUNT] AS FLOAT)/7) AS INFECTED_TOTAL_AVERAGE,
        [WEEKLY_COUNT] / (CAST([POPULATION] AS FLOAT)/100000) AS INFECTED_PER_100K,
        (CAST([WEEKLY_COUNT] AS FLOAT)/7) / (CAST([POPULATION] AS FLOAT)/100000) AS INFECTED_PER_100K_AVERAGE
    FROM 
       VWSINTER.ECDC_CASES
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) from VWSINTER.ECDC_CASES)
        AND [INDICATOR] = 'cases'
        AND COUNTRY_CODE IN (
        'AUT',
        'BEL',
        'BGR',
        'CHE',
        'CYP',
        'CZE',
        'DEU',
        'DNK',
        'ESP',
        'EST',
        'FIN',
        'FRA',
        'GBR',
        'GRC',
        'HRV',
        'HUN',
        'IRL',
        'ISL',
        'ITA',
        'LIE',
        'LTU',
        'LUX',
        'LVA',
        'MLT',
        'NLD',
        'NOR',
        'POL',
        'PRT',
        'ROU',
        'SVK',
        'SVN',
        'SWE'
    )
END;
GO

# **DATATINO CONFIGURATIONS**
---

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'ECDC_CASES',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

IF NOT EXISTS ( SELECT 1 FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] WHERE NAME LIKE @workflow_name )
BEGIN
    -- 1.1) SET ENVIRONMENTAL VARIABLES.....
    SET @is_active = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 1
        WHEN 'acceptance' THEN 1
        ELSE 1
    END;

    SELECT TOP(1)
        @workflow_id = workflows.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
    WHERE v_workflows.[NAME] = @workflow_name;

    SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
        @id = @workflow_id, 
        @workflow_name = @workflow_name,
        @description = @workflow_description,
        @schedule = '0 14 * * THU', -- At 01:00 PM, only on Thursday
        @active = @is_active;

    -- 2) UPSERT SOURCE(S).....
    DECLARE @source NVARCHAR(256),
            @source_id INT,
            @source_name NVARCHAR(256),
            @source_description NVARCHAR(256),
            @target_name VARCHAR(256),
            @location_type VARCHAR(50),
            @security_profile VARCHAR(50);

    -- 2.1) SET ENVIRONMENTAL VARIABLES.....
    SET @source = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/csv/data.csv'
        WHEN 'acceptance' THEN 'https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/csv/data.csv'
        ELSE 'https://opendata.ecdc.europa.eu/covid19/nationalcasedeath/csv/data.csv'
    END;

    SET @location_type = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'Web'
        WHEN 'acceptance' THEN 'Web'
        ELSE 'Web'
    END;

    SET @security_profile = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'N/A'
        WHEN 'acceptance' THEN 'N/A'
        ELSE 'N/A'
    END;

    -- 2.2) UPSERT TABLE SOURCE(S): STAGING......
    SET @source_name = CONCAT('SOURCE_', @workflow_name);
    SET @source_description = CONCAT('LOAD: ', @workflow_name);
    SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = 'country|country_code|continent|population|indicator|weekly_count|year_week|rate_14_day|cumulative_count|source',
        @target_columns = 'COUNTRY|COUNTRY_CODE|CONTINENT|POPULATION|INDICATOR|WEEKLY_COUNT|YEAR_WEEK|RATE_14_DAY|CUMULATIVE_COUNT|SOURCE',
        @target_name = @target_name,
        @source_type = 'CsvFile',
        @location_type = @location_type,
        @delimiter_type = 'Colon',
        @security_profile= @security_profile;

    -- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
    SET @source = CONCAT('dbo.SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');

    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = null,
        @target_columns = null,
        @target_name = null,
        @source_type = 'StoredProcedure',
        @location_type = 'N/A',
        @delimiter_type = 'N/A',
        @security_profile= @security_profile;

    -- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
    SET @source = CONCAT('dbo.SP_', @workflow_name, '_INTER_TO_DEST');
    SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_INTER_TO_DEST');
    SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO DEST');

    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = null,
        @target_columns = null,
        @target_name = null,
        @source_type = 'StoredProcedure',
        @location_type = 'N/A',
        @delimiter_type = 'N/A',
        @security_profile= @security_profile;

    -- 3) UPSERT PROCESS(ES).....
    DECLARE @process_id INT,
            @process_name NVARCHAR(256),
            @process_description NVARCHAR(256),
            @process_source_name VARCHAR(256);

    -- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
    SET @process_name = CONCAT('PROCESS_', @workflow_name);
    SET @process_description = CONCAT('LOAD: ', @workflow_name);
    SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;

    -- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO INTER......
    SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');
    SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;

    -- 3.3) UPSERT STORED PROCEDURE PROCESS(S): INTER TO DEST......
    SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_INTER_TO_DEST');
    SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO DEST');
    SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_INTER_TO_DEST');

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;

    -- 4) UPSERT DEPENDENC(Y)(IES).....
    DECLARE @dependency_id INT,
            @dependency_name NVARCHAR(256),
            @dependency_description NVARCHAR(256),
            @dependency_dataflow_name NVARCHAR(256),
            @dependency_process_name NVARCHAR(256);

    -- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES).....
    SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGE_TO_INTER');
    SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name);
    SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

    SELECT TOP(1)
        @dependency_id = dependencies.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
    WHERE dependencies.[NAME] = @dependency_name
        AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
        @id = @dependency_id,
        @dataflow_name = @dependency_dataflow_name,
        @dataflowtype_id = 2,
        @dependency_name = @dependency_process_name,
        @dependencytype_id = 2,
        @workflow_name = @workflow_name,
        @name = @dependency_name,
        @description = @dependency_description,
        @active = 1;

    -- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
    SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_INTER_TO_DEST');
    SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_INTER_TO_DEST');
    SET @dependency_process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
    SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

    SELECT TOP(1)
        @dependency_id = dependencies.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
    WHERE dependencies.[NAME] = @dependency_name
        AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
        @id = @dependency_id,
        @dataflow_name = @dependency_dataflow_name,
        @dataflowtype_id = 2,
        @dependency_name = @dependency_process_name,
        @dependencytype_id = 2,
        @workflow_name = @workflow_name,
        @name = @dependency_name,
        @description = @dependency_description,
        @active = 1;
    
END